In [ ]:
import os, subprocess
from pathlib import Path

In [ ]:
def get_current_dir_fun() -> str:
    return "exec('import os') or (str(globals()['_dh'][0]) if globals().get('_dh') else os.path.dirname(os.path.realpath(__file__)))"

def get_free_gpu_id() -> str:
    GPU_ID = subprocess.getoutput(
        'nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader | nl -v 0 | sort -nrk 2 | cut -f 1| head -n 1 | xargs')
    return GPU_ID

In [ ]:
__current_dir__ = eval(get_current_dir_fun())
mmseg_path = str(Path(__current_dir__).parent)
os.chdir(mmseg_path)
print(f'Change work directory to: {mmseg_path}')

os.environ['CUDA_VISIBLE_DEVICES'] = get_free_gpu_id()
print(f'USE GPU: {os.environ["CUDA_VISIBLE_DEVICES"]}')

In [ ]:
import time
import torch
from mmengine import Config
from mmengine.model.utils import revert_sync_batchnorm
from mmengine.registry import init_default_scope
from mmengine.runner import Runner, load_checkpoint
from mmseg.registry import MODELS
from matplotlib import pyplot as plt

### Download the pretrained model

In [ ]:
ckpt_file = 'pretrained_models/FFDN.pth'

In [ ]:
# Check dataset structure:
# data
# └── DocTamperV1
#     ├── unzip_files
#     │   ├── DocTamperV1-TrainingSet
#     │   ├── DocTamperV1-TestingSet
#     │   ├── DocTamperV1-FCD
#     │   └── DocTamperV1-SCD
#     ├── pks
#     │   ├── DocTamperV1-TestingSet_75.pk
#     │   ├── DocTamperV1-FCD_75.pk
#     │   └── DocTamperV1-SCD_75.pk
#     └── processed
#         ├── train.txt
#         ├── val.txt
#         ├── fcd.txt
#         └── scd.txt

assert os.path.exists('./data/DocTamperV1/processed/val.txt')
assert os.path.exists('./data/DocTamperV1/pks/DocTamperV1-TestingSet_75.pk')
assert os.path.exists('./data/DocTamperV1/unzip_files/DocTamperV1-TestingSet'), 'Dataset not found, please download the dataset first'

In [ ]:
# build dataloader

os.environ['compress_pk_file'] = './data/DocTamperV1/pks/DocTamperV1-TestingSet_75.pk'
os.environ['val_db_path'] ='./data/DocTamperV1/unzip_files/DocTamperV1-TestingSet'

config_file = 'work_config/FFDN/FFDN.py'
cfg = Config.fromfile(config_file)

init_default_scope(cfg.get('default_scope', 'mmseg'))
cfg.model.data_preprocessor.binary_seg = False
cfg.test_dataloader.batch_size = 1
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())

cfg.test_dataloader.dataset.ann_file="processed/val.txt"
cfg.model.pretrained = None
cfg.model.train_cfg = None
cfg.test_dataloader.batch_size = 1

In [ ]:
data_loader = Runner.build_dataloader(cfg.test_dataloader)

In [ ]:
# build model
model = MODELS.build(cfg.model)
if hasattr(model, 'auxiliary_head'):
    model.auxiliary_head = None
if torch.cuda.is_available():
    model = model.cuda()

model = revert_sync_batchnorm(model)

model.eval()
print(f'Create model with config: {config_file}')

In [ ]:
load_checkpoint(model, ckpt_file, map_location='cpu')

In [ ]:
data_iter = iter(data_loader) if globals().get('data_iter') is None else data_iter
example_data = next(data_iter)

example_data = model.data_preprocessor(example_data)
outs = model(example_data['inputs'], example_data['data_samples'], mode='predict')

gts = [data.gt_sem_seg.data for data in example_data['data_samples']]
segs = [out.pred_sem_seg.data for out in outs]

### Visualize the segmentation results

In [ ]:
input_image = example_data['inputs']['x'].cpu().squeeze(0).permute(1, 2, 0).numpy()
input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
gt_image = gts[0].cpu().squeeze(0).numpy()
seg_image = segs[0].cpu().squeeze(0).numpy()

# Create a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# Plot input image
ax1.imshow(input_image)
ax1.set_title('Input Image')
ax1.axis('off')

# Plot ground truth
ax2.imshow(gt_image, cmap='gray')
ax2.set_title('Ground Truth')
ax2.axis('off')

# Plot segmentation
ax3.imshow(seg_image, cmap='gray')
ax3.set_title('Segmentation')
ax3.axis('off')

plt.tight_layout()
plt.show()